In [ ]:
import pandas as pd
import glob
import os

import json

In [ ]:
#### MERGE ALL THE FILES IN ONE
# setting the path for joining multiple files
files = os.path.join("data-2/", "Kickstarter*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
print(df)


In [ ]:
#save merged raw dataframe as a file 
df.to_csv('data-2/data_raw.csv')

In [ ]:
#convert needed columns with dict content into columns
cols = ['category']
for col in cols:
    pd.json_normalize
    stdf = df[col].apply(json.loads)
    extracted = pd.DataFrame(stdf.tolist())
    extracted = extracted.add_prefix(f'{col}_')
    df = pd.concat([df, extracted], axis = 1)
    df = df.drop(col, axis = 1)

#drop non-convertable columns + columns which don't have any extra information 
# (photo, urls, category_urls)
col = ['creator', 'location', 'profile', 'photo', 'urls', 'category_urls']
df = df.drop(col, axis = 1)

In [ ]:
#save working dataframe as file
df.to_csv('data-2/working_data.csv')